# Landsat upgrade

1. create list of locations/times for satellite imagery download
2. download sentinel image and cut into standard 400 pix by 400 pix tiles
3. download matching landsat image
4. save figures to be used in GAN training model

In [124]:
## libraries
import sys
import datacube
from odc.ui import with_ui_cbk

# plotting
from matplotlib import pyplot as plt
# maths
import numpy as np
# save png images
import imageio
import csv

## DEA scripts
#sys.path.append("Scripts")

#from dea_datahandling import load_ard
#from DEAPlotting import rgb

from multiprocessing import Pool
from IPython.display import clear_output


## GLOBALS
# sentinel Analysis Ready Data
sentinel_product = "s2a_ard_granule"
sentinel_bands = ['nbart_red', 'nbart_green', 'nbart_blue']
# landsat granule
landsat_product = "ga_ls8c_ard_3"
landsat_bands = ['nbart_red', 'nbart_green', 'nbart_blue']

# how many white or black squares before we flag and remove?
# 5%
flag_pct = .05

# image name will be batchi/uid_product_time[flag].png
imgname = "batch_%1d/%04d_%s_%02d%s.png"
batch=1

In [125]:
## Datacube connection
dc = datacube.Datacube(app="Sentinel_2")

In [126]:
## CSV has list of points and dates to be pulled from sentinel
def get_batch(batch=batch, 
              csvfile = 'Points wgs rand 2k - Points wgs rand 2k.csv',
              maxcount = None):
    """
    READ 
    """
    count=0
    list_of_chips = []
    x_space = -0.051
    y_space = -0.034

    with open(csvfile) as csv_file:
        readCSV = csv.reader(csv_file, delimiter=',')
        for row in readCSV:
            # column 7 is 'batch', each batch (there are 4) has 2000 locations
            if row[7]==str(batch):
                # top left to bottom right lat/lon bounds used to request data
                x_min = float(row[3])
                y_min = float(row[4])
                x_max = float(row[3])-x_space
                y_max = float(row[4])+y_space
                uid = int(row[0])
                x=(x_min,x_max)
                y=(y_min,y_max)

                # request a month at a time
                year = int(row[5])
                month = int(row[6])
                date_str = "%4d-%02d"%(year,month)

                # save entries into a list
                list_of_chips.append([uid,date_str,x,y])

                # maybe we don't want all entries
                count+=1
                if maxcount is not None:
                    if count>maxcount:
                        break
    return list_of_chips
list_of_chips = get_batch(1)
for uid,t,x,y in list_of_chips[:2]:
    print(uid,":", t)
    print(x,y)
print(len(list_of_chips), " chips in the list")

7 : 2016-01
(141.1705998, 141.22159979999998) (-30.521397, -30.555397)
8 : 2019-10
(150.5310058, 150.5820058) (-35.18853004, -35.22253004)
491  chips in the list


In [127]:

## Show extra printouts?
verbose=False

In [128]:
def jesse_normalise(imgarr, divby=4000.):
    img = np.copy(imgarr)
    img[img<0] = 0
    img = (img * (255./divby)).astype(np.float)
    img[img>255] = 255
    img = np.rint(img).astype(np.uint8) # round to integer
    return img

def chipper(chip):
    """
        download and create image from sentinel and landsat products
        INPUT:
            chip: [[id,'YYYY-MM',(lon0,lon1),lat0,lat1],[...]]
    """
    flag = ""
    uid = chip[0]
    time = chip[1]
    x = chip[2]
    y = chip[3]
    
    ## Create a query object
    if verbose:
        print("INFO: chipper called:")
        print("INFO: x, y, time: ",x,y,time)
        
    query = {
        "x": x,
        "y": y,
        "time": time,
        "output_crs": "EPSG:3577",
        "resolution": (-10, 10),
        "group_by": "solar_day",
    }
    
    ## download ard granule
    ds = dc.load(
        product=sentinel_product,
        measurements = sentinel_bands,
        #progress_cbk = with_ui_cbk(),
        **query
    )
    
    if verbose:
        print(ds)
    
    # some months have multiple images
    time_step_count=0
    
    # Only get images if sentinel has some
    if len(ds.sizes)==0:
        print("WARNING: NO DATA FOR TIME RANGE",time)
    else:
        for time_step in ds.time:
            ## READ RGB into numpy array [y,x,channel]
            img = np.moveaxis(ds[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=time_step_count).to_array().values,0,2)
            if verbose:
                print("INFO: downloaded data details:")
                print("INFO: shape, type: ",img.shape, type(img))
                print("INFO: min, max: ", np.min(img), np.max(img))
            
            ## Normalise image to uint
            img = jesse_normalise(img)
            
            if np.sum(img==0) > flag_pct*np.size(img):
                flag="_black"
            if np.sum(img==255) > flag_pct*np.size(img):
                flag+="_white"
            
            ## Cut to 400x400
            n_y,n_x,_ = np.shape(img)
            dim_max=400
            assert n_x > dim_max, "ERR: X DIM IS < 400"
            assert n_y > dim_max, "ERR: Y DIM IS < 400"
            img_cut = img[:dim_max,:dim_max,:]
            
            ## how much of the lat/lon area are we cutting?
            x_cut = float(dim_max)/n_x
            y_cut = float(dim_max)/n_y
            
            ## apply same cut to our lat/lon bounding box
            new_x_1 = x[0] + (x[1]-x[0])*x_cut
            new_y_1 = y[0] + (y[1]-y[0])*y_cut
            
            if verbose:
                print("INFO: img after cutting:")
                print("INFO: shape before, after: ",img.shape, img_cut.shape)
                print("INFO: lons before, after: ", x, '->', x[0], new_x_1)
                print("INFO: lats before, after: ", y, '->', y[0], new_y_1)

            ## Save figure
            # no longer saving flagged images
            if flag == "":
                fname=imgname%(batch,uid,"sentinel",time_step_count,flag)
                imageio.imwrite(fname,img_cut)
                print("INFO: Saved figure: ",fname)
            flag=""
    
        ## download landsat equivalent
        # match landsat square to sentinel (need to fudge the edges a bit)
        # want slightly less lon1
        # and slighly more southern lat1
        xfudge=-0.00075 # ~75m
        yfudge=-0.0005 # ~50m
        query = {
            "x": (x[0], new_x_1+xfudge),
            "y": (y[0], new_y_1+yfudge),
            "time": time,
            "output_crs": "EPSG:3577",
            "resolution": (-25, 25),
            "group_by": "solar_day",
        }

        ## download ard granule
        #lsbands = ['Visible']
        ds2 = dc.load(
            product=landsat_product,
            measurements=landsat_bands,
            #progress_cbk=with_ui_cbk(),
            **query
        )

        if verbose:
            print(ds2)
        time_step_count=0
        if len(ds2.sizes)==0:
            print("WARNING: NO DATA FOR TIME RANGE",time)
        else:
            for time_step in ds2.time:
                lsimg = np.moveaxis(ds2[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=time_step_count).to_array().values,0,2)
                lsimg = jesse_normalise(lsimg)

                if np.sum(lsimg==0) > flag_pct*np.size(lsimg):
                    flag="_black"
                if np.sum(lsimg==255) > flag_pct*np.size(lsimg):
                    flag+="_white"
                
                # no longer saving flagged images
                if flag == "":
                    if verbose:
                        print("INFO: landsat img:", lsimg.shape, lsimg.shape)

                    lsfname=imgname%(batch,uid,"landsat",time_step_count,flag)
                    imageio.imwrite(lsfname,lsimg)
                    print("INFO: Saved figure: ",lsfname)
                time_step_count=+1
                flag=""

                

In [87]:
## Test chipper function
chipper(list_of_chips[0])

INFO: Saved figure:  images/0007_sentinel_00.png
INFO: Saved figure:  images/0007_landsat_01.png


In [129]:
## if arg is empty, use all available processes (but this is temperamental)
## there's like 16 processes available for use?
pool = Pool()
## run chipper function with list of chips  as input argument
## each function call can be done independently
pool.map(chipper, list_of_chips)
pool.close()
#pool.join()
#clear_output()
print("INFO:FINISHED")
print()
print()

INFO: Saved figure:  images/0807_sentinel_00.png
INFO: Saved figure:  images/0627_sentinel_00.png
INFO: Saved figure:  images/0678_sentinel_00.png
INFO: Saved figure:  images/0511_sentinel_00.png
INFO: Saved figure:  images/0474_sentinel_00.png
INFO: Saved figure:  images/0474_sentinel_00.png
INFO: Saved figure:  images/0604_sentinel_00.png
INFO: Saved figure:  images/0604_sentinel_00.png
INFO: Saved figure:  images/0918_sentinel_00.png
INFO: Saved figure:  images/0918_sentinel_00.png
INFO: Saved figure:  images/0559_sentinel_00.png
INFO: Saved figure:  images/0559_sentinel_00.png
INFO: Saved figure:  images/0559_sentinel_00.png
INFO: Saved figure:  images/0869_sentinel_00.png
INFO: Saved figure:  images/0869_sentinel_00.png
INFO: Saved figure:  images/0627_landsat_00.png
INFO: Saved figure:  images/0627_landsat_01.png
INFO: Saved figure:  images/0957_sentinel_00.png
INFO: Saved figure:  images/0957_sentinel_00.png
INFO: Saved figure:  images/0957_sentinel_00.png
INFO: Saved figure:  i

OperationalError: (psycopg2.OperationalError) SSL error: wrong version number

[SQL: SELECT agdc.dataset_type.id, agdc.dataset_type.name, agdc.dataset_type.metadata, agdc.dataset_type.metadata_type_ref, agdc.dataset_type.definition, agdc.dataset_type.added, agdc.dataset_type.added_by 
FROM agdc.dataset_type ORDER BY agdc.dataset_type.name ASC]

In [130]:
## Loop through images, remove any unmatched satellite pairs
from glob import glob
import os

sentinel_list0 = glob("batch%d/*_sentinel_*"%batch)
landsat_list0 = glob("batch%d/*_landsat_*"%batch)
sentinel_list0.sort()
landsat_list0.sort()

# cut off the _00.png from entries in list
sentinel_list = [entry[:-7] for entry in sentinel_list0]
landsat_list = [entry[:-7] for entry in landsat_list0]

print(len(sentinel_list), len(landsat_list))
print(sentinel_list[0])
for sentinel_file in sentinel_list:
    _, name = sentinel_file.split('/')
    uint, _ = name.split('_')
    if "images/%s_landsat"%uint not in landsat_list:
        to_remove="images/%s_sentinel*"%uint
        print("INFO: deleting ",to_remove)
        remove_list = glob(to_remove)
        for rmfile in remove_list:
            os.remove(rmfile)

for landsat_file in landsat_list:
    _, name = landsat_file.split('/')
    uint, _ = name.split('_')
    if "images/%s_sentinel"%uint not in sentinel_list:
        to_remove="images/%s_landsat*"%uint
        print("INFO: deleting ",to_remove)
        remove_list = glob(to_remove)
        for rmfile in remove_list:
            os.remove(rmfile) 



256 413
images/0007_sentinel
INFO: deleting  images/0016_sentinel*
INFO: deleting  images/0456_sentinel*
INFO: deleting  images/0473_sentinel*
INFO: deleting  images/0516_sentinel*
INFO: deleting  images/0527_sentinel*
INFO: deleting  images/0604_sentinel*
INFO: deleting  images/0612_sentinel*
INFO: deleting  images/0615_sentinel*
INFO: deleting  images/0722_sentinel*
INFO: deleting  images/0739_sentinel*
INFO: deleting  images/0750_sentinel*
INFO: deleting  images/0764_sentinel*
INFO: deleting  images/0822_sentinel*
INFO: deleting  images/0830_sentinel*
INFO: deleting  images/0852_sentinel*
INFO: deleting  images/0926_sentinel*
INFO: deleting  images/0980_sentinel*
INFO: deleting  images/0990_sentinel*
INFO: deleting  images/0993_sentinel*
INFO: deleting  images/1018_sentinel*
INFO: deleting  images/1028_sentinel*
INFO: deleting  images/1039_sentinel*
INFO: deleting  images/1044_sentinel*
INFO: deleting  images/1052_sentinel*
INFO: deleting  images/1057_sentinel*
INFO: deleting  image

In [144]:

# Zip the images folder
foldername = "batch%d"%batch
!tar cvzf zipname.tar.gz $foldername
clear_output()

# Manual Filtering

1. Download batch folder
2. Manually remove cloud affected and weird images
3. upload these into new folder batchx_filtered



In [149]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def show_images(uid):
    fnames_s = glob("batch%d/%04d_sen*"%(batch,uid))
    fnames_l = glob("batch%d/%04d_lan*"%(batch,uid))
    # Show uid images in a grid
    f,axes = plt.subplots(2,3,figsize=[16,14])
    
    ## Loop over landsat images
    for i,fname in enumerate(fnames_l):
        plt.sca(axes[0,i])
        plt.imshow(mpimg.imread(fname))
        index = batch_1_full_list.index(fname)
        plt.title("%s (folder index:%d)"%(fname,index))
    
    ## loop over sentinel images
    for i,fname in enumerate(fnames_s):
        plt.sca(axes[1,i])    
        plt.imshow(mpimg.imread(fname))
        index = batch_1_full_list.index(fname)
        plt.title("%s (folder index:%d)"%(fname,index))
    
    for ax in axes.flatten():
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels("")
        ax.set_yticklabels("")
    plt.tight_layout()

show_images(7)

NameError: name 'batch' is not defined

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# plt.imshow(mpimg.imread("fname"))

In [147]:
#print(list_of_chips[-5:])

<img src="" />